<a href="https://colab.research.google.com/github/ashuonnet/ailearn/blob/main/haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
#from haystack.telemetry import tutorial_running

#tutorial_running(3)


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [4]:
!git clone https://github.com/cisco-ncs/chat

Cloning into 'chat'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 38.88 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [5]:
!pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=5f0482a80d8c6b373662ae69d73bcf0989d8ec91915092d6f6c4e194b56e5406
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [6]:
mkdir chat/config-guides/txt

In [9]:
import os
from tika import parser


pdf_path = "chat/config-guides"
count = 0
for filename in os.listdir(pdf_path):
    if filename.endswith("pdf"):
        # Your code comes here such as
        pdf_file_path = pdf_path+"/"+filename
        print(pdf_file_path)
        raw = parser.from_file(pdf_file_path)

        txt_path = pdf_path+"/txt/"+str(count)+".txt"
        file1=open(txt_path,"w")
        file1.writelines(raw['content'])
        count = count+1

chat/config-guides/b-qos-cg-ncs5500-78x.pdf
chat/config-guides/b-multicast-cg-ncs55k-78x.pdf
chat/config-guides/b-l3vpn-cg-ncs5500-78x.pdf
chat/config-guides/b-mpls-cg-ncs5500-78x.pdf
chat/config-guides/b-ip-addresses-cg-ncs5500-78x.pdf
chat/config-guides/b-routing-cg-ncs5500-78x.pdf
chat/config-guides/b-segment-routing-cg-ncs5500-78x.pdf
chat/config-guides/b-bgp-cg-ncs5500-78x.pdf
chat/config-guides/b-l2vpn-cg-ncs5500-78x.pdf


In [10]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [11]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch


In [12]:
import time

time.sleep(30)


In [13]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")


INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


In [14]:
#from haystack.utils import fetch_archive_from_http

doc_dir = "chat/config-guides/txt"

#fetch_archive_from_http(
#    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip",
#    output_dir=doc_dir,
#)


In [15]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])


In [17]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)


INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing: 100%|██████████| 9/9 [00:04<00:00,  2.04docs/s]


{'documents': [<Document: {'content': "\n\nSegment Routing Configuration Guide for Cisco NCS 5500 Series Routers, IOS XR Release 7.8.x\n\nSegment Routing Configuration Guide for Cisco NCS 5500 Series\nRouters, IOS XR Release 7.8.x\nFirst Published: 2022-11-30\n\nLast Modified: 2023-03-13\n\nAmericas Headquarters\nCisco Systems, Inc.\n170 West Tasman Drive\nSan Jose, CA 95134-1706\nUSA\nhttp://www.cisco.com\nTel: 408 526-4000\n\n800 553-NETS (6387)\nFax: 408 527-0883\n\nTHE SPECIFICATIONS AND INFORMATION REGARDING THE PRODUCTS IN THIS MANUAL ARE SUBJECT TO CHANGE WITHOUT NOTICE. ALL STATEMENTS,\nINFORMATION, AND RECOMMENDATIONS IN THIS MANUAL ARE BELIEVED TO BE ACCURATE BUT ARE PRESENTED WITHOUT WARRANTY OF ANY KIND,\nEXPRESS OR IMPLIED. USERS MUST TAKE FULL RESPONSIBILITY FOR THEIR APPLICATION OF ANY PRODUCTS.\n\nTHE SOFTWARE LICENSE AND LIMITED WARRANTY FOR THE ACCOMPANYING PRODUCT ARE SET FORTH IN THE INFORMATION PACKET THAT SHIPPED WITH\nTHE PRODUCT AND ARE INCORPORATED HEREIN BY TH

In [18]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [19]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [20]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])


In [21]:
prediction = querying_pipeline.run(
    #query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
    query = "how to configure srv6?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print(prediction)


Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.42s/ Batches]

{'query': 'how to configure srv6?', 'no_ans_gap': 2.616826295852661, 'answers': [<Answer {'answer': 'exit\nRouter', 'type': 'extractive', 'score': 0.59415602684021, 'context': 'fig-isis-srv6)# locator myLocBestEffort\nRouter(config-isis-srv6-loc)# exit\nRouter(config-isis-srv6)# locator myLocLowLat\nRouter(config-isis-srv6-loc)#', 'offsets_in_document': [{'start': 1023, 'end': 1034}], 'offsets_in_context': [{'start': 70, 'end': 81}], 'document_ids': ['d1bb36722133f75d6b9f810efb06e0d8'], 'meta': {'_split_id': 68, '_split_overlap': [{'range': [0, 132], 'doc_id': 'fe12c9dfa1d178878e586442ef36f126'}, {'range': [1191, 1533], 'doc_id': '19166d3dec5524f1f9209fe192501821'}]}}>, <Answer {'answer': 'Router', 'type': 'extractive', 'score': 0.38939112424850464, 'context': 'r-te-segment-lists)# segment-list p1_r8_1\nRouter(config-sr-te-sl)# srv6\nRouter(config-sr-te-sl-srv6)# no topology-check\n\nRunning Configuration\n\nsegmen', 'offsets_in_document': [{'start': 1208, 'end': 1214}], 'offsets_in_co

In [22]:
from pprint import pprint

pprint(prediction)


{'answers': [<Answer {'answer': 'exit\nRouter', 'type': 'extractive', 'score': 0.59415602684021, 'context': 'fig-isis-srv6)# locator myLocBestEffort\nRouter(config-isis-srv6-loc)# exit\nRouter(config-isis-srv6)# locator myLocLowLat\nRouter(config-isis-srv6-loc)#', 'offsets_in_document': [{'start': 1023, 'end': 1034}], 'offsets_in_context': [{'start': 70, 'end': 81}], 'document_ids': ['d1bb36722133f75d6b9f810efb06e0d8'], 'meta': {'_split_id': 68, '_split_overlap': [{'range': [0, 132], 'doc_id': 'fe12c9dfa1d178878e586442ef36f126'}, {'range': [1191, 1533], 'doc_id': '19166d3dec5524f1f9209fe192501821'}]}}>,
             <Answer {'answer': 'Router', 'type': 'extractive', 'score': 0.38939112424850464, 'context': 'r-te-segment-lists)# segment-list p1_r8_1\nRouter(config-sr-te-sl)# srv6\nRouter(config-sr-te-sl-srv6)# no topology-check\n\nRunning Configuration\n\nsegmen', 'offsets_in_document': [{'start': 1208, 'end': 1214}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['6

In [24]:
querying_pipeline.save_to_yaml('my_pipeline.yml')

In [25]:
pwd

'/content'